<a href="https://colab.research.google.com/github/dave502/NLP/blob/main/lesson_13/nlp_hw_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
%%capture
!pip install transformers

In [24]:
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import csv
import random

In [25]:
device = torch.device("cuda")#("cpu")#

###ГЕНЕРАЦИЯ ТВИТОВ

#####Загрузка данных

In [4]:
%%capture
!rm -r data data.zip
!wget https://www.dropbox.com/s/s04sixz0366tj8r/data.zip
!unzip data.zip
!rm data.zip

In [5]:
!ls data

train.csv  val.csv


In [6]:
!head -n 5 data/train.csv

id,text,class
0,"@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.",0
1,"RT @GalyginVadim: Ребята и девчата!
Все в кино!!! ""Вот Это Любовь!""
Сегодня! Завтра!  И потом!)))))


In [7]:
with open('data/train.csv', 'r') as train_csv:
  reader = csv.reader(train_csv)
  next(reader, None) # skip header
  train_list = [rows[1] for rows in reader]
train_list[:5]

['@alisachachka не уезжаааааааай. :(❤ я тоже не хочу, чтобы ты уезжала.',
 'RT @GalyginVadim: Ребята и девчата!\nВсе в кино!!! "Вот Это Любовь!"\nСегодня! Завтра!  И потом!)))))\n#вотэтолюбовь',
 'RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретвит((((( RT',
 'RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :)',
 '@KarineKurganova @Yess__Boss босапопа есбоса не боится мороза и .......))']

#####Препроцессинг текста

In [8]:
def preprocessing(text_lines:list, sep:str='')->list:
  """
  функция педобработки данных
  убираются сайты, символы, цифры, стоп-слова и токены с единичной длинной
  """  
  # регулярное выражение для очистки текста
  url_pattern = "(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])"
  smile_pattern = "(:D|D:)"
  usernames_pattern = "(@[\w]*|RT @[\w]*)"
  onlywords_pattern = "[^\w\b]|[^A-Za-zА-ЯЁа-яё]"
  pattern = f"{smile_pattern}|{url_pattern}|{usernames_pattern}|{onlywords_pattern}"

  preprocessed = [re.sub(pattern, ' ', line) for line in text_lines]
  preprocessed = [(re.sub(r'\s+', ' ', line)+sep).lstrip() for line in preprocessed]

  return preprocessed

In [9]:
sep = '\n***\n'
train = preprocessing(train_list, sep)

#####Инициализация модели GPT

In [ ]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

#####Отбор нескольких строк из обучающей выборки

In [11]:
prefix = sep + ''.join(random.sample(train, 5))
print(prefix)


***
Ты вообще одна с парнями фоткалась что плохого то 
***
посмотрела у нее были две бэшки на одной на другой еще украинская бэшка в декабре есть 
***
блин приди и уберись мне 
***
На то он и мозг чтобы иногда нас удивлять х 
***
пельмешечная идея витает в воздухе хммм 
***



#####Токенизация текста

In [12]:
tokens = tokenizer(prefix, return_tensors='pt')
tokens = {k: v.to(model.device) for k,v in tokens.items()}
end_token_id = tokenizer.encode('***')[0]

In [13]:
end_token_id

24633

#####Генерация текста моделью

In [21]:
# количество полученных токенов
size = tokens['input_ids'].shape[1]
# генарация текста
output = model.generate(**tokens,
                        eos_token_id=end_token_id,
                        do_sample=False,
                        max_length=size+128,
                        repetition_penalty=4.2,
                        temperature=0.7,
                        num_beams=10,
                        )
decoded = tokenizer.decode(output[0])
result = decoded[len(prefix):]
print(result)

Setting `pad_token_id` to `eos_token_id`:24633 for open-end generation.


я не знаю как тебе объяснить, но я очень хочу чтоб ты была со мной. Я так скучаю по тебе 
 ***
у меня сегодня день рожденья а это значит что мы будем отмечать его где-нибудь в другом месте х 
***


###АНАЛИЗ ТЕКСТА И ОТВЕТЫ ПО ТЕКСТУ

In [1]:
%%capture
!pip install deeppavlov

In [3]:
from deeppavlov import build_model, configs

Инициализация бертовой модели

In [6]:
model_dp = build_model(configs.squad.squad_ru_bert, download=True)

2022-12-06 15:32:09.420 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/v1/squad/squad_ru_torch_bert.tar.gz to /root/.deeppavlov/squad_ru_torch_bert.tar.gz
INFO:deeppavlov.core.data.utils:Downloading from http://files.deeppavlov.ai/v1/squad/squad_ru_torch_bert.tar.gz to /root/.deeppavlov/squad_ru_torch_bert.tar.gz
100%|██████████| 1.74G/1.74G [01:28<00:00, 19.6MB/s]
2022-12-06 15:33:38.964 INFO in 'deeppavlov.core.data.utils'['utils'] at line 276: Extracting /root/.deeppavlov/squad_ru_torch_bert.tar.gz archive into /root/.deeppavlov/models
INFO:deeppavlov.core.data.utils:Extracting /root/.deeppavlov/squad_ru_torch_bert.tar.gz archive into /root/.deeppavlov/models


Downloading:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/642 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from 

Анализируемый текст

In [7]:
text = \
"""
Далеко-далеко, в не замеченных картографами складках давно вышедшего из моды Западного Спирального Рукава Галактики, затерялась крохотная, никому не интересная желтая звезда.
Вокруг нее, на расстоянии примерно девяноста восьми миллионов миль, обращается ничтожная зелено-голубая планетка, обитатели которой все еще очень похожи на своих предков-обезьян – достаточно сказать, что электронные часы до сих пор считаются у них чудом техники.
У этой планеты есть – вернее, была – одна проблема: большинство живущих на ней людей только и делали, что страдали, так как не находили в жизни счастья. Рождалось множество решений, но почти все они сводились к перераспределению маленьких зеленых клочков бумаги – что само по себе весьма странно, так как кто-кто, а маленькие зеленые клочки бумаги никаких страданий не испытывали, ибо счастья не искали.
Решение все никак не находилось, и планета полнилась озлобленными людьми, для большинства из которых ощущение несчастья было постоянным, и даже электронные часы не скрашивали им жизнь.
Постепенно распространялось и крепло убеждение, что все несчастья пошли с того, как люди спустились с деревьев на землю. А кое-кто даже полагал, что ошибка была совершена еще раньше – с деревьями тоже нечего было связываться и вообще незачем было вылезать из океана.
И вот как-то в четверг, после дождя, спустя почти две тысячи лет после того, как одного человека приколотили гвоздями к дереву за то, что он призывал хотя бы иногда, просто для разнообразия, относиться друг к другу по-хорошему, некая девушка, сидя в одиночестве за столиком маленького кафе в Рикмансворте, вдруг додумалась, в чем была вся загвоздка и каким образом мир все-таки можно сделать обителью счастья и покоя. На сей раз дело в шляпе, все непременно получится – и никаких гвоздей и приколачиваний живых людей к деревьям и прочим предметам!
К сожалению, не успела она дойти до телефона, чтобы поделиться с кем-нибудь своим открытием, произошла чудовищно нелепая катастрофа, и решение было навсегда потеряно.
Впрочем, история, которую я собираюсь вам поведать, не о девушке.
Это история о чудовищно нелепой катастрофе и некоторых ее последствиях.
Также это история о книге. О книге под названием «Автостопом по Галактике: Путеводитель вольного странника», в обиходе именуемой просто «Путеводитель» – книге, написанной не на Земле, никогда на Земле не публиковавшейся и до катастрофы неизвестной ни одному землянину. Тем не менее это замечательная книга.
Пожалуй, это самое замечательное из всех изданий грандиозной Издательской корпорации Малой Медведицы – также неизвестной ни одному землянину.
Книга пользуется огромным спросом, оставив далеко позади «Межзвездный сборник советов по домоводству»; раскупается гораздо быстрее, чем «Еще 53 способа скоротать время в невесомости», и вызывает еще больше споров и толков, чем философская трилогия «Где Господь сбился с пути», «Основные ошибки Господа Бога» и «Да кто он в конце концов такой, этот Господь Бог?!»
В просвещенных и расслабленных цивилизациях Внешнего Восточного Кольца Галактики «Путеводитель» вытеснил даже великую «Большую Галактическую Энциклопедию» в качестве общепризнанного кладезя мудрости и знаний. Ибо, несмотря на обилие пропусков и вопиюще искаженной, а то и просто-напросто апокрифической информации, «Путеводитель» обладает двумя важнейшими преимуществами перед «Энциклопедией».
Во-первых, он дешевле. Во-вторых, на титульном листе крупными буквами выведены сакраментальные слова: «НЕ ПАНИКУЙ».
"""

#####Проверка анализа текста и генерации ответов

In [11]:
model_dp([text], ['где затерялась звезда?'])

[['в не замеченных картографами складках давно вышедшего из моды Западного Спирального Рукава Галактики'],
 [16],
 [1.0]]

In [12]:
model_dp([text], ['что написано на листе?'])

[['«НЕ ПАНИКУЙ»'], [1381], [0.9999987483024597]]

In [13]:
model_dp([text], ['о чем книга?'])

[['о чудовищно нелепой катастрофе и некоторых ее последствиях'],
 [12],
 [0.9999992847442627]]

###ДИАЛОГ С ПОМОЩЬЮ GPT 

Функция генерации ответа по списку предыдущих фраз

In [47]:
def continue_dialog(texts):

  # вставка префиксов, определяющих участников диалога
  prefix = '\nx:'
  for i, t in enumerate(texts):
    prefix += t
    prefix += '\nx:' if i%2 == 1 else '\ny:'
  # токенизация дилога
  tokens = tokenizer(prefix, return_tensors='pt')
  tokens = {k: v.to(model.device) for k, v in tokens.items()}
  end_token_id = tokenizer.encode('\n')[0]
  # добавление следующей фразы 
  size = tokens['input_ids'].shape[1]
  output = model.generate(**tokens,
                          eos_token_id=end_token_id,
                          do_sample=True,
                          max_length=size+128,
                          repetition_penalty=3.2,
                          temperature=1,
                          num_beams=3,
                          length_penalty=0.01,
                          pad_token_id=tokenizer.eos_token_id)
  # декодирование полученных токенов
  decoded = tokenizer.decode(output[0])
  # возврат последней фразы из диалога
  result = decoded[len(prefix):]
  return result.strip()

In [51]:
start_phrase = input('Начните диалог:\n') # получаем первую фразу от пользователя
history = [start_phrase]
while True:
  response = continue_dialog(history[-10:])   # получаем ответ от модели на основании последних 10-ти фраз
  next_sentence = input(response + '\n') # выводи ответ и получаем следующую фразу от пользователя
  if next_sentence.lower()=='пока': break # если пользователь прощается - выходим
  history.append(response) # запись новых фраз в историю диалога
  history.append(next_sentence)

Начните диалог:
Добрый вечер!
Здравствуйте.
Как Вас зовут?
Лариса.
Как дела, Лариса?
Нормально.
Вы не знаете какая сечас погода?
Нет.
А время сейчас какое?
Четверг.
Нет, сегодня вторник.
Понятно.
Вы не расположены к диалогу, да?
Да.
Тогда не буду Вас больше беспокоить
Почему?
У меня много дел
Какие дела?
Мне пора готовить ужин
Простите, а что это значит?
Вам не понять
Что-что?
Пока
